In [5]:
import polars as pl
from datetime import datetime


In [6]:
df = pl.read_csv("./Data/JuvenileCaseInfo.csv")

In [11]:
df.with_columns(
    pl.col("FiledDate").map_elements(lambda x: datetime.strptime(x, "%m/%d/%Y"))
).sort(by = "FiledDate", descending=True).filter( pl.col("County") == "Douglas" ).select(["CaseID", "FiledDate"])["CaseID"].to_list()[:5]

['D 01 JV 24 0001745',
 'D 01 JV 24 0001742',
 'D 01 JV 24 0001743',
 'D 01 JV 24 0001740',
 'D 01 JV 24 0001736']